In [64]:
import openai
import os

In [2]:
# !pip install praw

     ---------------------------------------- 0.0/189.4 kB ? eta -:--:--
     ------------------------------------- 189.4/189.4 kB 11.2 MB/s eta 0:00:00


In [12]:
import praw

In [65]:
client_id = "your_client_id"
client_secret = "your_client_secret"

os.environ['OPENAI_API_KEY'] = 'YOUR__API_KEY'
openai.api_key = os.getenv('OPENAI_API_KEY')


In [15]:
reddit = praw.Reddit(client_id=client_id, # import os  os.environ['REDDIT_CLIENT']
                    client_secret=client_secret,
                    user_agent = "sentiment analysis test"
                    )

In [28]:
for submission in reddit.subreddit("stocks").hot(limit=5):
    print(submission.title) # reddit.com/r/stocks

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
/r/Stocks Weekend Discussion Saturday - Apr 08, 2023
A $1.5 Trillion Wall of Debt Is Looming for US Commercial Properties
Beyond Meat stock analysis and valuation - A worthless company?
Is Costco over valued?


In [38]:
subreddit_stocks = reddit.subreddit("stocks")

In [39]:
subreddit_stocks.title

'Stocks - Investing and trading for all'

In [40]:
subreddit_stocks.accounts_active

3629

In [47]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title) # post have unique id (post.id)
    submission = reddit.submission(post.id)
    
    # print top 2 comments per title submission
    counter = 0
    for comments in submission.comments:
        if comments.body == '[deleted]':
            pass
        counter += 1
        if counter == 2:
            break
    
    

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
/r/Stocks Weekend Discussion Saturday - Apr 08, 2023
A $1.5 Trillion Wall of Debt Is Looming for US Commercial Properties
Beyond Meat stock analysis and valuation - A worthless company?
Is Costco over valued?


In [48]:
###  Perform Sentiment Analysis via OpenAI

In [55]:
def get_titles_and_comments(subreddit="stocks", limit=6, num_comments=3, skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    title_and_comments = {}

    for counter, post in enumerate(subreddit.hot(limit=limit)):
        if counter < skip_first:
            continue

        counter += (1 - skip_first)

        title_and_comments[counter] = ""
        # PRAW
        submission = reddit.submission(post.id)
        title = post.title

        title_and_comments[counter] += "Title: " + title + "\n\n"
        title_and_comments[counter] += "Comments: \n\n"

        comment_counter = 0
        for comment in submission.comments:
            if not comment.body == "[deleted]":
                title_and_comments[counter] += comment.body + "\n"
                comment_counter += 1
            if comment_counter == num_comments:
                break
    return title_and_comments


In [56]:
titles_and_comments = get_titles_and_comments()

In [57]:
titles_and_comments

{1: 'Title: A $1.5 Trillion Wall of Debt Is Looming for US Commercial Properties\n\nComments: \n\nImo these commercial real estate owners live in a different reality. \n\nI keep seeing empty properties since before the pandemic. These just stay there… some without being sold and some without the price being reduced thinking someone will eventually come and pay the outrageous price.\n\nAnd now that many are being converted to regular housing they also want to either charge a lot or don’t pay for the transformation so again they rather hold it empty for YEARS.  And this is stuff from before the pandemic.\nI live in AZ and it’s not uncommon to see multiple tilt slab construction without any tenants now. From what I understand many of these are planned 2 years ahead so thank you COVID boom/bust. \nI agree, this could be ugly for commercial and industrial properties.\nSo how does one short it?\n',
 2: "Title: Beyond Meat stock analysis and valuation - A worthless company?\n\nComments: \n\nL

In [59]:
# Create the final prompt

def create_prompt(title_and_comments):
    task = "Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive , negative, or neutral. If no ticker or company is mentioned write 'No company mentioned'\n\n"
    return task + title_and_comments
    

In [62]:
print(create_prompt(titles_and_comments[1])) # sending this into GPT engine

Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive , negative, or neutral. If no ticker or company is mentioned write 'No company mentioned'

Title: A $1.5 Trillion Wall of Debt Is Looming for US Commercial Properties

Comments: 

Imo these commercial real estate owners live in a different reality. 

I keep seeing empty properties since before the pandemic. These just stay there… some without being sold and some without the price being reduced thinking someone will eventually come and pay the outrageous price.

And now that many are being converted to regular housing they also want to either charge a lot or don’t pay for the transformation so again they rather hold it empty for YEARS.  And this is stuff from before the pandemic.
I live in AZ and it’s not uncommon to see multiple tilt slab construction without any tenants now. From what I understand many of these are planned 2 years ahead so tha

In [67]:
for key, title_with_comments in titles_and_comments.items():
    prompt = create_prompt(title_with_comments)
    
    response = openai.Completion.create(engine='text-davinci-003', 
                                        prompt=prompt, 
                                        max_tokens=256, 
                                        temperature=0, 
                                        top_p=1.0
                                       )
    print(title_with_comments)
    print(f"Sentiment Report from OpenAI: {response['choices'][0]['text']}")
    print("---------------------------------------------")

Title: A $1.5 Trillion Wall of Debt Is Looming for US Commercial Properties

Comments: 

Imo these commercial real estate owners live in a different reality. 

I keep seeing empty properties since before the pandemic. These just stay there… some without being sold and some without the price being reduced thinking someone will eventually come and pay the outrageous price.

And now that many are being converted to regular housing they also want to either charge a lot or don’t pay for the transformation so again they rather hold it empty for YEARS.  And this is stuff from before the pandemic.
I live in AZ and it’s not uncommon to see multiple tilt slab construction without any tenants now. From what I understand many of these are planned 2 years ahead so thank you COVID boom/bust. 
I agree, this could be ugly for commercial and industrial properties.
So how does one short it?

Sentiment Report from OpenAI: 
No company mentioned.
---------------------------------------------
Title: Beyond 